<a href="https://colab.research.google.com/github/LaineWishart/Degree/blob/master/Laine_EDA_visualisation_functions_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
a

# COVID-19 Visualisation Functions

Created by Laine, 2020-11-05

In [ ]:

import pandas as pd
import numpy as np
import io
import matplotlib.pyplot as plt
from typing import List, Dict
!pip install sigfig
import sigfig
import scipy
import plotly.graph_objects as go
import plotly.io as pio
from datetime import datetime
from plotly.subplots import make_subplots
import scipy.stats as sp
import random 
import datetime
import plotly.express as px
%load_ext google.colab.data_table
from google.colab import data_table




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My\ Drive

/content/drive/My Drive


# Test dataset

The test dataset will have data for the following columns:


1. country
2. time
3. new_deaths: number of new deaths per day
4. new_tests: number of new tests conducted per day
5. new_cases: number of new cases conducted per day 

The variables 'new_deaths', 'new_tests', 'new_cases' were generated from a Poisson distribution. A poisson distribution was selected because we observed in our exploratory data analysis that all these variables follow a Poisson distribution. Lambda was selected according to the minimum and maximum numbers recorded for those variables. NaN values were incorporate for 10% of the values for 'new_deaths', 'new_tests', and 'new_cases'. The dates generated randomly from a range of December (when first COVID-19 case was reported) to today. 

In [ ]:
# helper functions to create the artificial dataset
def generate_poisson_distribution(n_distributions, distribution_size, lam_list):

  arr = np.zeros(n_distributions*size)

  counter = 0

  for i in range(0, n_distributions):
      dat = np.random.poisson(lam=lam_list[i], size=size)
      arr[counter:counter+size] = dat
      counter += size
  return arr
  

# adapted from https://www.kite.com/python/answers/how-to-generate-a-random-date-between-two-dates-in-python
start_date = datetime.date(2019, 12, 1)
end_date = datetime.date(2020, 11,4 )

def generate_random_date(start_date, end_date):

  time_between_dates = end_date - start_date
  days_between_dates = time_between_dates.days
  random_number_of_days = random.randrange(days_between_dates)
  random_date = start_date + datetime.timedelta(days=random_number_of_days)

  return random_date


In [ ]:
# Generate the artificial dataset

size = 20
percent_null = 0.1
n_distributions = 5

# generate list of random dates
date = [0]*(size*n_distributions)

for i in range(0, size*n_distributions):
  date[i] = generate_random_date(start_date, end_date)

date = np.array(date)

# generate list of country names 
country = np.array(['canada'*size, 'australia'*size, 'united states'*size, 'united kingdom'*size, 'singapore'*size])
country = ['canada']*size + ['australia']*size + ['united states']*size + ['united_kingdom']*size + ['singapore']*size
country = np.array(country)

# list of reasonable lambda values for each variable
lambda_deaths_per_day = random.sample(range(0, 10), 5)
lambda_tests_per_day = random.sample(range(250, 2000), 5)
lambda_cases_per_day = random.sample(range(10, 6000), 5)


# new deaths
new_deaths = generate_poisson_distribution(5, size, lambda_deaths_per_day)

# new cases 
new_cases = generate_poisson_distribution(5, size, lambda_cases_per_day)

# new tests
new_tests = generate_poisson_distribution(5, size, lambda_tests_per_day)

# combine data into one dataframe 
index = np.array(list(range(0,size*n_distributions)))
test_data= pd.DataFrame({'country': country, 'time': date, 'new_deaths': new_deaths, 'new_cases': new_cases, 'new_tests': new_tests})
nan_mat = np.random.random(test_data.loc[:, ['new_deaths', 'new_cases', 'new_tests']].shape)<0.1

test_data.loc[:, ['new_deaths', 'new_cases', 'new_tests']]= test_data.loc[:, ['new_deaths', 'new_cases', 'new_tests']].mask(nan_mat)

test_data['time'] = pd.to_datetime(test_data['time'], dayfirst = False)
test_data['month'] = pd.DatetimeIndex(test_data['time']).month

## Summary statistics function

#Functions included here
- a function that generates summary statistics for each country and variable
- generates barcharts that shows the number months that fall into bins of x count
- a function that shows how the variable changes over time, and number of tests - showing the line colour as more or less blury?
- create a function that shows the relationship between multiple variables over time

In [ ]:

# replace None with the variable to be selected
variable = 'new_deaths'
country_str = 'Australia'
data = test_data
nbins = 4

country = country_str.lower()

 # assign a colour to each country
colour_dict = {'australia': '#636EFA' , 'canada': '#EF553B', 'india': '#00CC96', 'singapore': '#FFA15A', 'united_kingdom': '#FF97FF', 'united_states': '#B6E880'}

# get the colour for the selected country
colour = colour_dict[country]


# this will be used in titles and axis labels
str_var  = variable.replace('new_', '')


# select variable from data 
var_df = data[data.loc[:, 'country']==country].loc[:, ['month', variable]]

# get the sum of the variable for each month
sum_per_month = var_df.groupby('month').sum()

# generate plot labels
labels = {variable: "Total {var} per month".format(var = str_var), 'count': 'Number of months in range'}

# generate the figure
fig = px.histogram(sum_per_month, x = variable, nbins=nbins, marginal = "rug", labels = labels, title = "Number of months so far with with the same total {var} count".format(var = str_var.replace('s', '')))

fig.show()

In [ ]:

def get_barchart(country_str, data, variable, nbins):
  """Generates barcharts that shows the number months with the same total count of variable."""
  # convert country to lowercase
  country = country_str.lower()
  str_var  = variable.replace('new_', '') # use this for titles and label axes

  # select relevant data
  var_df =   data[data.loc[:, 'country']==country].loc[:, ['month', variable]]

  # group by month and calculate the sum
  sum_per_month = var_df.groupby('month').sum()

  # define x and y labels
  labels = {variable: "Total {var} per month".format(var = str_var), 'count': 'Number of months in range'}

  # generate the figure
  fig = px.histogram(sum_per_month, x = variable, nbins=nbins, marginal = "rug", labels = labels, title = "{country}: Number of months so far with with the same total {var} count".format(country = country, var = str_var.replace('s', '')))

  fig.show()  

# example of how to use the function 
get_barchart(country, data, variable, nbins)

## Variable over time function

In [ ]:
#
def plot_one_variable_over_time(data, country, y):
 
  str_y = y.replace('new_', '') # use this for title and label axes

  # select relevant data
  var_df =   data[data.loc[:, 'country']==country].loc[:, ['time', y]]
  
  # create a figure
  fig = go.Figure()

  # gemerate a line plot
  fig.add_trace(go.Scatter (
      x = data.loc[:, x], 
      y = data.loc[:, y], 
      name = y
  ))

    # update the figure layout
  fig.update_layout(
      title="Change in COVID-19 {y} per day for {country} over time".format(y = str_y, country = country),
      xaxis_title= "Time" ,
      yaxis_title= "Count per day", 
      autosize=False,
      width=1200,
      height=800,
      )
  
  fig.show()

# example of how it should work
#plot_one_variable_over_time(data, country, y1)

NameError: ignored

## Two variable function

In [ ]:
data = test_data
country = "Australia"
y1 = 'new_deaths'
y2 = 'new_cases'
x = 'time'

str_y1 = y1.replace('new_', '')
str_y2 = y2.replace('new_', '')

var_df =   data[data.loc[:, 'country']==country].loc[:, [x, y1, y2]]
fig = go.Figure()

fig.add_trace(go.Scatter (
    x = data.loc[:, x], 
    y = data.loc[:, y1], 
    name = y1
))

fig.add_trace(go.Scatter (
    x = data.loc[:, x], 
    y = data.loc[:, y2], 
    name = y2
))

  # update the figure layout
fig.update_layout(
    title="Change in COVID-19 {y1} and {y2} per day for {country} over time".format(y1 = str_y1, y2 = str_y2, country = country),
    xaxis_title= "Time" ,
    yaxis_title= "Count per day", 
    autosize=False,
    width=1200,
    height=800,
    )



In [ ]:
def plot_two_variables_over_time(data, country, y1, y2):

  # use these for titles and axis labels
  str_y1 = y1.replace('new_', '')
  str_y2 = y2.replace('new_', '')

  # select relevant data
  var_df =   data[data.loc[:, 'country']==country].loc[:, [x, y1, y2]]
  
  # create a figure
  fig = go.Figure()

  # add the first line plot
  fig.add_trace(go.Scatter (
      x = data.loc[:, x], 
      y = data.loc[:, y1], 
      name = y1
  ))

  # add the second line plot
  fig.add_trace(go.Scatter (
      x = data.loc[:, x], 
      y = data.loc[:, y2], 
      name = y2
  ))

  # update the figure layout
  fig.update_layout(
      title="Change in COVID-19 {y1} and {y2} per day for {country} over time".format(y1 = str_y1, y2 = str_y2, country = country),
      xaxis_title= "Time" ,
      yaxis_title= "Count per day", 
      autosize=False,
      width=1200,
      height=800,
      )
  
  fig.show()

# example of how the function should be used
plot_two_variables_over_time(data, country, y1, y2)

In [ ]:
def plot_one_var_over_time(country,y1_str):

  # change the input parameters to representations of columns
  country2 = country.lower().replace(' ', '_')
  y1_str = y1.lower().replace(' ', '_')

  # put all countries and make countries the row index
  all_countries = pd.concat(list_of_countries, keys = ['australia', 'canada', 'india', 'singapore', 'united_kingdom', 'united_states'])

  all_countries['month'] = pd.DatetimeIndex(all_countries['time']).month
  
  # change NaN values to None so plotly can plot gaps
  all_countries.loc[:, [y1]] = all_countries.loc[:, [y1]].where(pd.notnull(all_countries.loc[:, [y1]]), None)

  # get mortality metrics from df for selected country
  vars_df = all_countries.loc[country2][['time', y1]]

  # make time the index
  vars_df.index = vars_df['time']
  vars_df.drop('time', axis=1, inplace=True)

  # calculate the mean, sd, and n for each var
  y_mean = vars_df.astype('float').resample('M').mean()
  y_sd = vars_df.astype('float').resample('M').std()
  y_n = vars_df.astype('float').resample('M').count()
  y_n.astype(int)

  # put all the stats together
  y_stats = pd.concat([y_mean.T, #.T transposes the dataframe
                                      y_sd.T,
                                      y_n.T],
                                      keys = ['mean', 'sd', 'count']).reset_index()

  # replace index with multi-level index
  y_stats.sort_values(['level_1', 'level_0'], inplace=True)
  multi_idx = pd.MultiIndex.from_frame(y_stats.loc[:, ['level_1', 'level_0']], names = ["mortality_metric", "stat"])
  y_stats= y_stats.set_index(multi_idx).drop(['level_0', 'level_1'], axis=1)

  # round to nearest decimal place
  y_stats = round(y_stats, 2)

  # get the selected mortality metric
  selected_y_stats = y_stats.loc[y1]

  # assign a colour to each country
  colour_dict = {'australia': '#636EFA' , 'canada': '#EF553B', 'india': '#00CC96', 'singapore': '#FFA15A', 'united_kingdom': '#FF97FF', 'united_states': '#B6E880'}

  # get the colour for the selected country
  colour = colour_dict[country2]

  # assign values to x and y variables
  x = all_countries.loc[country2]['time']
  y = all_countries.loc[country2][y1]

  # create a plotly figure
  fig = make_subplots(
      rows = 2, cols=1, 
      shared_xaxes = False, 
      vertical_spacing = 0.09, 
      specs = [[{"type": "scatter"}], 
              [{"type": "table"}]], 
  )

  # add a table
  fig.add_trace(
      go.Table(header = dict(values = ["",
                                      "December 2019",
                                      "January 2020",
                                      "February 2020",
                                      "March 2020",
                                      "April 2020",
                                      "May<br>2020",
                                      "June 2020", 
                                      "July<br>2020", 
                                      "August 2020", 
                                      "September 2020", 
                                      "October 2020", 
                                      "November 2020"], 
                                line_color='darkslategray',
      fill_color='aliceblue',
      align=['left','center'],
      font=dict(color='black', size=12),
      height=40), 
              cells = dict(
                  values = [["Count", "Mean", "Sd"]] + [selected_y_stats[k].tolist() for k in selected_y_stats.columns[0:]], 
                      line_color='darkslategray',
      fill=dict(color=['aliceblue', 'white']),
      align=['left', 'center'],
      font_size=12,
      height=30)),  
      row = 2, col=1
  )

  # add a line plot
  fig.add_trace(go.Scatter(
      x=x,
      y=y,
      name='fig1',
      line=dict(color=colour, width=2)
  ), row = 1, col=1)

  # update the figure layout
  fig.update_layout(
      title="COVID-19 {y1} per day for {country} over time".format(y1 = y1, country = country),
      xaxis_title= "Time" ,
      yaxis_title= y1_str, 
      autosize=False,
      width=1200,
      height=800,
      )

  fig.show(renderer = 'colab')

plot_mortality_rate(country, 'mortality_rate')

NameError: ignored

# Test functions on real data

# Data cleaning and Engineering

Next, import the data required for analysis. We originally explored data from a number of sources, but for the sake of making direct comparisons using the data we have elected to conduct all of our analysis using a single dataset that contains all the information we require. This data has been sourced from Our World in Data (https://ourworldindata.org/coronavirus).

In [ ]:
# Created by Laine, 2020-10-30
# import csv file straight from Our World in Data's github repo (updates automatically)
raw_data = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv')

Now, we explore the shape of the data and the variables in the dataset.

In [ ]:
# Created by Julia, 2020-10-29
print("Shape of dataset: ", raw_data.shape)
raw_data.head()

Shape of dataset:  (55033, 49)


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,tests_per_case,positive_rate,tests_units,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,ABW,North America,Aruba,2020-03-13,2.0,2.0,NaN,NaN,0.0,NaN,18.733,18.733,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN
1,ABW,North America,Aruba,2020-03-19,NaN,NaN,0.286,NaN,NaN,0.0,NaN,NaN,2.676,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.33,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN
2,ABW,North America,Aruba,2020-03-20,4.0,2.0,0.286,NaN,0.0,0.0,37.465,18.733,2.676,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.33,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN
3,ABW,North America,Aruba,2020-03-21,NaN,NaN,0.286,NaN,NaN,0.0,NaN,NaN,2.676,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.44,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN
4,ABW,North America,Aruba,2020-03-22,NaN,NaN,0.286,NaN,NaN,0.0,NaN,NaN,2.676,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.44,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN


We can see that this dataset is quite large. It contains 53087 rows and 41 columns. We can also see a number of missing values, which we will leave as NaN. These missing values are one source of uncertainty in the data - we do not know if there is a systematic reason that they are missing, and we do not know whether having this data would affect our analysis. Keeping in mind that we do not have a complete dataset will remind us to be careful about drawing conclusions.

We will now filter the data down to the variables required for our analysis - determining how deadly COVID-19 is in 6 countries: Australia, Canada, India, Singapore, the United Kingdom, and the United States.

In [ ]:
# Created by Julia, 2020-10-29
# First, create a datetime variable

raw_data['time'] = pd.to_datetime(raw_data['date'], dayfirst = True)

# function to create separate datasets for each country

def select_country(raw_data, iso_code, select_cols):
  raw_data = raw_data
  iso_code = iso_code
  data = raw_data.loc[:, select_cols]
  data = data[data.loc[:, 'iso_code'] == iso_code ]

  return data

We then use the function to tidy the data, creating individual dataframes for each country. Each dataframe will contain corresponding variables.

In [ ]:
# Created by Laine, 2020-10-30

def select_country(raw_data, iso_code, select_cols):
  raw_data = raw_data
  iso_code = iso_code
  data = raw_data.loc[:, select_cols]
  data = data[data.loc[:, 'iso_code'] == iso_code ]

  return data

In [ ]:

# Created by Julia 2020-10-29
# Adapted by Laine 2020-10-30 -- added select_cols, converted country names to lower case for easier coding 
select_cols = ['time',
                'iso_code',  #left in so we can easily check the data
                'new_cases',
                'total_cases',
                'new_deaths',
                'total_deaths',
                'new_tests',
                'total_tests',
                'population',
                'median_age',
                'aged_65_older',
                'aged_70_older',
                'gdp_per_capita',
                'cardiovasc_death_rate',
                'diabetes_prevalence'
                ]

australia = select_country(raw_data, 'AUS', select_cols)
canada = select_country(raw_data, 'CAN', select_cols)
india = select_country(raw_data, 'IND', select_cols)
singapore = select_country(raw_data, 'SGP', select_cols)
united_kingdom = select_country(raw_data, 'GBR', select_cols)
united_states = select_country(raw_data, 'USA', select_cols)

We now have 6 individual tidy dataframes, one for each country of interest: Australia, Canada, India, Singapore, the United Kingdom, and the United States. Each dataframe now has 303 rows and 15 columns. The rows are observations, and the columns are: time, ISO code, new cases, total cases, new deaths, total deaths, new tests, total tests, population, median age, people aged 65+, people aged 70+, GDP per capita, cardiovascular death rate, and diabetes prevalence.

## Data transformations 


We created two variables called 'case_fatality_rate': (confirmed deaths/confirmed_cases)*100 and 'mortality_rate' (confirmed deaths / total population)*100 000

We created a function to calculate the case fatality rate, mortality rate per 100 000 people and tests per 100 000 people per day for each country, and then insert these 3 new variables into each dataframe.

These functions were then tested and combined into one function to streamline the process of creating and adding CFR, MR and test rate into each dataframe.

In [ ]:
# created by Julia, 2020-10-29
# edited by Laine, 2020-10-30: new vars weren't appearing in data after this function was applied, so i changed it a bit (your original code is commented out)
# function to calculate CFR, MR and test rate and insert into each dataframe

def insert_variables(data):
  """function to calculate case fatality rate (CFR), mortality rate (MR) and test rate and insert into each dataframe. """

  CFR = round((data['new_deaths']/data['new_cases'])*100,1) #1 significant figure

  MR = round((data['new_deaths']/data['population'])*100000,1) #1 significant figure

  test_rate = round((data['new_tests']/data['population'])*100000,1) #1 significant figure

  data['case_fatality_rate'] = CFR
  data['mortality_rate'] = MR
  data['test_rate'] = test_rate

  return data

We used this function to create the 3 new variables for each dataframe.

In [ ]:
# created on 2020-10-30 by Laine
# purpose: to reduce code duplication in the cell below
list_of_countries = [australia, canada, india, singapore, united_kingdom, united_states]

for c in list_of_countries:
  c = insert_variables(c)

all_countries = pd.concat(list_of_countries, keys = ['australia', 'canada', 'india', 'singapore', 'united_kingdom', 'united_states'])

all_countries.rename(columns = {'new_deaths': 'deaths', 'new_tests': 'tests', 'new_cases': 'cases'}, inplace=True)

In [ ]:
all_countries2

time iso_code  ...  mortality_rate  test_rate
australia     2591  2019-12-31      AUS  ...             0.0        NaN
              2592  2020-01-01      AUS  ...             0.0        NaN
              2593  2020-01-02      AUS  ...             0.0        NaN
              2594  2020-01-03      AUS  ...             0.0        NaN
              2595  2020-01-04      AUS  ...             0.0        NaN
...                        ...      ...  ...             ...        ...
united_states 51484 2020-11-02      USA  ...             0.1      207.0
              51485 2020-11-03      USA  ...             0.2      201.6
              51486 2020-11-04      USA  ...             0.3        NaN
              51487 2020-11-05      USA  ...             0.3        NaN
              51488 2020-11-06      USA  ...             0.4        NaN

[1872 rows x 18 columns]

# Test functions

In [ ]:
plot_two_variables_over_time(all_countries, 'australia', 'new_deaths', 'new_cases')

In [ ]:
def plot_one_var_over_time(data, country, y):
 
  str_y = y.replace('new_', '') # use this for title and label axes

  # select relevant data
  var_df =   data[data.loc[:, 'country']==country].loc[:, ['time', y]]
  
  # create a figure
  fig = go.Figure()

  # gemerate a line plot
  fig.add_trace(go.Scatter (
      x = data.loc[:, x], 
      y = data.loc[:, y], 
      name = y
  ))

    # update the figure layout
  fig.update_layout(
      title="Change in COVID-19 {y} per day for {country} over time".format(y = str_y, country = country),
      xaxis_title= "Time" ,
      yaxis_title= "Count per day", 
      autosize=False,
      width=1200,
      height=800,
      )
  
  fig.show()

# example of how it should work
plot_one_variable_over_time(all_countries, country, y1)

# Functions in one cell

In [ ]:
# functions to calculate summary statistics

def select_vars(data, vars):
    """Selects desired variable from data."""
    selected_data = data.loc[:, vars]
    return selected_data 

def change_class(value):
    """Converts numpy integer values to python integer."""
    if isinstance(value, np.integer) or isinstance(value, np.float):
        value = int(value)
    else:
        value = value
    return value 

def apply_sig_fig(dict, sig_fig):
    """Applies specified number of significant figures to values in a dictionary."""
    new_dict = {}
    for i,j in dict.items():
        new_dict[i] = sigfig.round(
            change_class(j),
            sig_fig)
    return new_dict

## function to compute summary statistics 
def get_summary_stats(data, sig_fig):
    """Calculates summary statistics with correct number of significant figures."""
    mean = data.mean()
    median = data.median()
    std = data.std()
    total = data.sum()
    min_ = data.min()
    max_ = data.max()
    summary_stats = {'mean': mean, 'median': median, 'sd': std, 'total': total, 'min': min_, 'max': max_}
    summary_stats = apply_sig_fig(summary_stats, sig_fig = sig_fig)
    sum_stats_df = pd.DataFrame(data = list(summary_stats.values()) ,index= list(summary_stats.keys()))
    return sum_stats_df.T

def get_var_summary_stats(str_country, data, variable, sig_fig = 1):
  data = data[data.loc[:, 'country']==str_country].loc[:, variable]

  country = str_country.lower()

  sum_stats = get_summary_stats(data, sig_fig).T
  return data_table.DataTable(sum_stats, min_width= "3")
  

def get_barchart(country_str, data, variable, nbins):
  """Generates barcharts that shows the number months with the same total count of variable."""
  # convert country to lowercase
  country = country_str.lower()
  str_var  = variable.replace('_', ' ') # use this for titles and label axes

  # select relevant data
  var_df =   data[data.loc[:, 'country']==country].loc[:, ['month', variable]]

  # group by month and calculate the sum
  sum_per_month = var_df.groupby('month').sum()

  # define x and y labels
  labels = {variable: "Total {var} per month".format(var = str_var), 'count': 'Number of months in range'}

  # generate the figure
  fig = px.histogram(sum_per_month, x = variable, nbins=nbins, marginal = "rug", labels = labels, title = "{country}: Number of months so far with with the same total {var} count".format(country = country, var = str_var))

  fig.show()  


def plot_one_variable_over_time(country, data, y):
 
  str_y = y.replace('new_', '') # use this for title and label axes

  # select relevant data
  var_df =   data[data.loc[:, 'country']==country].loc[:, ['time', y]]
  
  # create a figure
  fig = go.Figure()

  # gemerate a line plot
  fig.add_trace(go.Scatter (
      x = data.loc[:, "time"], 
      y = data.loc[:, y], 
      name = y
  ))

    # update the figure layout
  fig.update_layout(
      title="Change in COVID-19 {y} per day for {country} over time".format(y = str_y, country = country),
      xaxis_title= "Time" ,
      yaxis_title= "Count per day", 
      autosize=False,
      width=1200,
      height=800,
      )
  
  fig.show()

def plot_two_variables_over_time(country, data, y1, y2):

  # use these for titles and axis labels
  str_y1 = y1.replace('new_', '')
  str_y2 = y2.replace('new_', '')

  # select relevant data
  var_df =   data[data.loc[:, 'country']==country].loc[:, ['time', y1, y2]]
  
  # create a figure
  fig = go.Figure()

  # add the first line plot
  fig.add_trace(go.Scatter (
      x = data.loc[:, 'time'], 
      y = data.loc[:, y1], 
      name = y1
  ))

  # add the second line plot
  fig.add_trace(go.Scatter (
      x = data.loc[:, 'time'], 
      y = data.loc[:, y2], 
      name = y2
  ))

  # update the figure layout
  fig.update_layout(
      title="Change in COVID-19 {y1} and {y2} per day for {country} over time".format(y1 = str_y1, y2 = str_y2, country = country),
      xaxis_title= "Time" ,
      yaxis_title= "Count per day", 
      autosize=False,
      width=1200,
      height=800,
      )
  
  fig.show()

def plot_one_var_over_time(country,y1_str):

  # change the input parameters to representations of columns
  country2 = country.lower().replace(' ', '_')
  y1= y1_str.lower().replace(' ', '_')

  # put all countries and make countries the row index
  list_of_countries = [australia, canada, india, singapore, united_kingdom, united_states]
  all_countries = pd.concat(list_of_countries, keys = ['australia', 'canada', 'india', 'singapore', 'united_kingdom', 'united_states'])

  # add a month variable
  all_countries['month'] = pd.DatetimeIndex(all_countries['time']).month

  # change NaN values to None so plotly can plot gaps
  all_countries.loc[:, [y1]] = all_countries.loc[:, [y1]].where(pd.notnull(all_countries.loc[:, [y1]]), None)

  # get vars from df for selected country
  vars_df = all_countries.loc[country2][['time', 'month', y1]]

  # make time the index
  vars_df.index = vars_df['time']
  vars_df.drop('time', axis=1, inplace=True)

  # calculate the mean, sd, and n for each var
  y_mean = vars_df.astype('float').resample('M').mean()
  y_sd = vars_df.astype('float').resample('M').std()
  y_n = vars_df.astype('float').resample('M').count()
  y_n.astype(int)

  # put all the stats together
  y_stats = pd.concat([y_mean.T, #.T transposes the dataframe
                                      y_sd.T,
                                      y_n.T],
                                      keys = ['mean', 'sd', 'count']).reset_index()

  # replace index with multi-level index
  y_stats.sort_values(['level_1', 'level_0'], inplace=True)
  multi_idx = pd.MultiIndex.from_frame(y_stats.loc[:, ['level_1', 'level_0']], names = ["mortality_metric", "stat"])
  y_stats= y_stats.set_index(multi_idx).drop(['level_0', 'level_1'], axis=1)

  # round to nearest decimal place
  y_stats = round(y_stats, 2)

  # get the selected mortality metric
  selected_y_stats = y_stats.loc[y1]

  # assign a colour to each country
  colour_dict = {'australia': '#636EFA' , 'canada': '#EF553B', 'india': '#00CC96', 'singapore': '#FFA15A', 'united_kingdom': '#FF97FF', 'united_states': '#B6E880'}

  # get the colour for the selected country
  colour = colour_dict[country2]

  # assign values to x and y variables
  x = all_countries.loc[country2]['time']
  y = all_countries.loc[country2][y1]

  # group by month and calculate the sum for the histogram
  sum_per_month = vara_df.groupby('month').sum()
  
  # create a plotly figure
  #fig = make_subplots(
  #    rows = 2, cols=1, 
  #    shared_xaxes = False, 
  #    vertical_spacing = 0.09, 
  #    specs = [[{"type": "scatter"}], 
  #            [{"type": "table"}]], 
  #)

  fig = make_subplots(
    rows=2, cols=2,
    specs=[[{}, {"rowspan": 2}],
           [{}, None]],
    subplot_titles=("First Subplot","Second Subplot", "Third Subplot"))

  # add a table
  fig.add_trace(
      go.Table(header = dict(values = ["",
                                      "December 2019",
                                      "January 2020",
                                      "February 2020",
                                      "March 2020",
                                      "April 2020",
                                      "May<br>2020",
                                      "June 2020", 
                                      "July<br>2020", 
                                      "August 2020", 
                                      "September 2020", 
                                      "October 2020", 
                                      "November 2020"], 
                                line_color='darkslategray',
      fill_color='aliceblue',
      align=['left','center'],
      font=dict(color='black', size=12),
      height=40), 
              cells = dict(
                  values = [["Count", "Mean", "Sd"]] + [selected_y_stats[k].tolist() for k in selected_y_stats.columns[0:]], 
                      line_color='darkslategray',
      fill=dict(color=['aliceblue', 'white']),
      align=['left', 'center'],
      font_size=12,
      height=30)),  
      row = 2, col=1
  )

  # add a line plot
  fig.add_trace(go.Scatter(
      x=x,
      y=y,
      name='fig1',
      line=dict(color=colour, width=2)
  ), row = 1, col=1)



  # update the figure layout
  fig.update_layout(
      title="COVID-19 {y1} per day for {country} over time".format(y1 = y1, country = country),
      xaxis_title= "Time" ,
      yaxis_title= y1_str, 
      autosize=False,
      width=1200,
      height=800,
      )

  fig.show(renderer = 'colab')

In [ ]:

#def get_barchart(country_str, data, variable, nbins):
  #"""Generates barcharts that shows the number months with the same total count of variable."""
  # convert country to lowercase
  #country = country_str.lower()
  #str_var  = variable.replace('_', ' ') # use this for titles and label axes

  # select relevant data
  #var_df =   data[data.loc[:, 'country']==country].loc[:, ['month', variable]]

  # group by month and calculate the sum
  #sum_per_month = var_df.groupby('month').sum()

  # define x and y labels
  #labels = {variable: "Total {var} per month".format(var = str_var), 'count': 'Number of months in range'}

  # generate the figure
  #fig = px.histogram(sum_per_month, x = variable, nbins=nbins, marginal = "rug", labels = labels, title = "{country}: Number of months so far with with the same total {var} count".format(country = country, var = str_var))



country = 'australia'
y1_str = 'new_cases'

# change the input parameters to representations of columns
country2 = country.lower().replace(' ', '_')
y1= y1_str.lower().replace(' ', '_')

# put all countries and make countries the row index
list_of_countries = [australia, canada, india, singapore, united_kingdom, united_states]
all_countries = pd.concat(list_of_countries, keys = ['australia', 'canada', 'india', 'singapore', 'united_kingdom', 'united_states'])

all_countries['month'] = pd.DatetimeIndex(all_countries['time']).month

# change NaN values to None so plotly can plot gaps
all_countries.loc[:, [y1]] = all_countries.loc[:, [y1]].where(pd.notnull(all_countries.loc[:, [y1]]), None)

# get vars from df for selected country
var_df = all_countries.loc[country2][['time', 'month', y1]]

# group by month and calculate the sum
sum_per_month = vara_df.groupby('month').sum()

# define x and y labels
labels = {variable: "Total {var} per month".format(var = str_var), 'count': 'Number of months in range'}

fig = make_subplots(
  rows=2, cols=2,
  specs=[[{}, {"rowspan": 2}],
          [{}, None]],
  subplot_titles=("First Subplot","Second Subplot", "Third Subplot"))


fig.add_trace(go.Histogram(x = sum_per_month.loc[:, y1], xbins=dict(start=np.min(sum_per_month.loc[:, y1]), size=1, end=np.max(sum_per_month.loc[:, y1]))))


NameError: ignored

In [ ]:
def get_monthly_totals(data, country, variable_name):
  """Returns monthly totals for a given variable."""

  # convert country to lowercase
  country = country.lower().replace(" ", "_")
  
  # get the index values
  multi_index_vals =  list(data.index.get_level_values(level=0).unique())

  # check the time variable is in datetime format
  assert np.issubdtype(data.loc[:, 'time'], np.datetime64), '"time" column dtype should be datetime64[ns]'

  # check the country is in the multi-index value list
  assert country in multi_index_vals, "{country} not in the index.".format(country=country)

  # check the input variable is in the columns
  assert variable_name in data.columns, "Input variable not found in dataframe columns. "

  # add a month column to the data
  data['month'] = pd.DatetimeIndex(data['time']).month

  # calculate the monthly totals for the given variable
  monthly_totals_df =  all_countries.loc[country][['month', variable_name]].groupby('month').sum()

  return monthly_totals_df

get_monthly_totals(all_countries, 'australia', 'new_deaths')

,new_deaths
month,
1,0.0
2,0.0
3,19.0
4,71.0
5,12.0
6,2.0
7,85.0
8,422.0
9,271.0


In [ ]:
def get_monthly_stats(data, country, variable_name):

  # convert country to lowercase
  country = country.lower().replace(" ", "_")
  
  # get the index values
  multi_index_vals =  list(data.index.get_level_values(level=0).unique())

   # check the time variable is in datetime format
  assert np.issubdtype(data.loc[:, 'time'], np.datetime64), '"time" column dtype should be datetime64[ns]'

  # check the country is in the multi-index value list
  assert country in multi_index_vals, "{country} not in the index.".format(country=country)

  # check the input variable is in the columns
  assert variable_name in data.columns, "Input variable not found in dataframe columns. "
  
  # get vars from df for selected country
  data_subset = data.loc[country][['time', variable_name]]

  # make time the index
  data_subset.index = data_subset['time']
  data_subset.drop('time', axis=1, inplace=True)

  # calculate the mean, sd, and n for each var
  mean = data_subset.astype('float').resample('M').mean().T
  sd = data_subset.astype('float').resample('M').std().T
  n = data_subset.astype('float').resample('M').count().astype(int).T
  med = data_subset.astype('float').resample('M').median().T
  max = data_subset.astype('float').resample('M').max().T
  min = data_subset.astype('float').resample('M').min().T
  
  # put all the stats together
  monthly_stats_list = pd.concat([n,
                       max,
                       min, 
                       med,
                       mean, 
                       sd ],
                       keys = [
                              'count',
                              'max',
                              'min', 
                              'median', 
                              'mean', 
                              'standard deviation'
                               ]
                      ).reset_index()

  # replace index with multi-level index
  multi_idx = pd.MultiIndex.from_frame(monthly_stats_list.loc[:, ['level_0']], names = ["stat"])
  monthly_stats_list= monthly_stats_list.set_index(multi_idx).drop(['level_0', 'level_1'], axis=1)
  monthly_stats_list.sort_values(['stat'], inplace=True)

  # round to nearest decimal place
  monthly_stats_list = round(monthly_stats_list, 1)

  return monthly_stats_list

get_monthly_stats(all_countries, 'australia', 'new_deaths')

time,2019-12-31 00:00:00,2020-01-31 00:00:00,2020-02-29 00:00:00,2020-03-31 00:00:00,2020-04-30 00:00:00,2020-05-31 00:00:00,2020-06-30 00:00:00,2020-07-31 00:00:00,2020-08-31 00:00:00,2020-09-30 00:00:00,2020-10-31 00:00:00,2020-11-30 00:00:00
stat,,,,,,,,,,,,
count,1.0,31.0,29.0,31.0,30.0,31.0,30.0,31.0,31.0,30.0,31.0,6.0
max,0.0,0.0,0.0,3.0,7.0,2.0,1.0,13.0,25.0,59.0,6.0,0.0
mean,0.0,0.0,0.0,0.6,2.4,0.4,0.1,2.7,13.6,9.0,0.8,0.0
median,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,13.0,6.5,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
standard deviation,NaN,0.0,0.0,0.9,1.8,0.6,0.3,3.4,5.4,11.8,1.5,0.0


In [ ]:
def visualise_one_var(data, country, variable_name, nbins = 1):
  # convert country to lowercase
  country = country.lower().replace(" ", "_")
  
  # get the index values
  multi_index_vals =  list(data.index.get_level_values(level=0).unique())

   # check the time variable is in datetime format
  assert np.issubdtype(data.loc[:, 'time'], np.datetime64), '"time" column dtype should be datetime64[ns]'

  # check the country is in the multi-index value list
  assert country in multi_index_vals, "{country} not in the index.".format(country=country)

  # check the input variable is in the columns
  assert variable_name in data.columns, "Input variable not found in dataframe columns. "

  # get the monthly totals for the histogram 
  monthly_totals = get_monthly_totals(data, country, variable_name)

  # get monthly summary stats for the table 
  monthly_stats = get_monthly_stats(data, country, variable_name)

  # assign a colour to each country
  colour_dict = {'australia': '#636EFA' , 'canada': '#EF553B', 'india': '#00CC96', 'singapore': '#FFA15A', 'united_kingdom': '#FF97FF', 'united_states': '#B6E880'}

  # get the colour for the selected country
  colour = colour_dict[country]

  # assign values to x and y variables
  x = data.loc[country]['time']
  y = data.loc[country][variable_name]

  subplot_titles = ("Daily {var} over time".format(var = variable_name.replace("_", " ")),
                    "Number of months with same monthly {var}".format(var = variable_name.replace("_", " ")),
                    "Monthly summary statistics for COVID-19 {var}".format(var=variable_name.replace("_", " "))) 

  fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"type": "scatter"}, {"type": "histogram"}],
           [{"colspan": 2,  "type": "table"}, None]],
    subplot_titles= subplot_titles, 
    vertical_spacing = 0.15, 
    horizontal_spacing = 0.15)

  # add a table
  fig.add_trace(
      go.Table(header = dict(values = ["",
                                      "December 2019",
                                      "January 2020",
                                      "February 2020",
                                      "March 2020",
                                      "April 2020",
                                      "May<br>2020",
                                      "June 2020", 
                                      "July<br>2020", 
                                      "August 2020", 
                                      "September 2020", 
                                      "October 2020", 
                                      "November 2020"], 
                                line_color='darkslategray',
      fill_color='aliceblue',
      align=['left','center'],
      font=dict(color='black', size=12),
      height=50), 
              cells = dict(
                  values = [[ 'n days with data',
                              'max',
                              'min', 
                              'median', 
                              'mean', 
                              'standard deviation'
                               ]] + [monthly_stats[k].tolist() for k in monthly_stats.columns[0:]], 
                      line_color='darkslategray',
      fill=dict(color=['aliceblue', 'white']),
      align=['left', 'center'],
      font_size=12,
      height=20)),  
      row = 2, col=1
  )

  # add a line plot
  fig.add_trace(go.Scatter(
      x=x,
      y=y,
      name='fig1',
      line=dict(color=colour, width=2)
  ), row = 1, col=1)

  # add histogram
  fig.add_trace(go.Histogram(name=None,
       y = monthly_totals.loc[:, variable_name],
      nbinsy = nbins, 
      ybins=dict(
          start=np.min(monthly_totals.loc[:, variable_name]),
          end=np.max(monthly_totals.loc[:, variable_name])
                 ), 
                 marker = dict(color = colour)
      ), row = 1, col = 2)
  
  # update the figure layout
  fig.update_layout(
      title="COVID-19 {var} in {country}".format(var = variable_name.replace("_", " "), country = country),
      autosize=True,
      width=1500,
      height=800,
      template = "plotly_white",
      showlegend=False
      )

  fig.update_yaxes(title_text="Daily case fatality rate", row=1, col=1)
  fig.update_xaxes(title_text="Number of months", row=1, col=2)

  fig.update_xaxes(title_text = "Time", row=1, col=1)
  fig.update_yaxes(title_text = "Monthly {var}".format(var = variable_name.replace("_", " ")), row=1, col=2)

  fig.show(renderer = 'colab')

plot_one_var_over_time(all_countries, 'canada', 'case_fatality_rate', nbins=10)

In [ ]:
def analyse_two_variables(data, country, first_variable, second_variable):

  # convert country to lowercase
  country = country.lower().replace(" ", "_")
  
  # get the index values
  multi_index_vals =  list(data.index.get_level_values(level=0).unique())

  # check the time variable is in datetime format
  assert np.issubdtype(data.loc[:, 'time'], np.datetime64), '"time" column dtype should be datetime64[ns]'

  # check the country is in the multi-index value list
  assert country in multi_index_vals, "{country} not in the index.".format(country=country)

  # check the input variables are in the columns
  assert first_variable in data.columns, "First input variable not found in dataframe columns. "
  assert second_variable in data.columns, "Second input variable not found in dataframe columns. "

  # assign a colour to each variable
  colour_dict = {'deaths': '#EF553B' , 'cases': '#636EFA', 'tests': '#00CC96', 'mortality_rate': '#FFA15A', 'case_fatality_rate': '#FF97FF'}

  subplot_titles = ("COVID-19 {first_variable} and {second_variable} in {country}".format(first_variable = first_variable.replace("_", " "), second_variable = second_variable.replace("_", " "), country = country))
  fig = make_subplots(rows = 1, cols =1, 
                      subplot_titles = subplot_titles)

  # select data for x and ys
  x = data.loc[country]['time']
  y1 = data.loc[country][first_variable]
  y2 = data.loc[country][second_variable]

  # specify colours for each variable
  y1_colour = colour_dict[first_variable]
  y2_colour = colour_dict[second_variable]

  fig.add_trace(go.Scatter(
    x=x,
    y=y1,
    name='Daily {first_variable}'.format(first_variable = first_variable.replace("_", " ")),
    line=dict(color=y1_colour, width=2) 

), row = 1, col=1)
  
  fig.add_trace(go.Scatter(
  x=x,
  y=y2,
  name='Daily {second_variable}'.format(second_variable = second_variable.replace("_", " ")),
  line=dict(color=y2_colour, width=2)
), row = 1, col=1)

  fig.update_layout(
      autosize=True,
      width=1500,
      height=800,
      template = "plotly_white",
      showlegend=True
      )
  
  fig.update_yaxes(title_text="Count", row=1, col=1)
  fig.update_xaxes(title_text="Time", row=1, col=1)

  fig.show()

analyse_two_variables(all_countries, 'canada', 'deaths', 'cases')

In [ ]:
"deaths": "dot", "cases": "dash", ""